In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.12.3


In [3]:
import pickle
import pandas as pd

In [9]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/anaconda3/envs/langchain/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [95]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet')


In [96]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [97]:
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
X_val = dv.transform(val_dicts)
y_pred_val = lr.predict(X_val)
lr.intercept_

23.197129450882166

In [86]:
import numpy as np
std_dev = np.std(y_pred_val)


In [87]:
std_dev

6.247500973564958

In [33]:
year = 2023
month = 3
taxi_type = "yellow"

In [92]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [93]:
output_file = f'/Users/amrit/Documents/Projects/DataTalkClub/cohort/mlops-Zoomcamp/notebooks/hw-4/data/{taxi_type}/{year:04d}_{month:02d}_result.parquet'

In [94]:
df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [99]:
print(f'Mean predicted duration: {y_pred_val.mean():.3f}')

Mean predicted duration: 14.243


In [42]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet')

In [48]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet')


In [ ]:
df_val_april = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
